In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]

load nci


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [ ]:
from joblib import Parallel, delayed
from tqdm import tqdm

n_kfold = 1
n_jobs = 50  # 並列数


def process_iteration(dim, target_index, seed, args):
    """各反復処理をカプセル化した関数"""
    if dim:
        if drug_sum[target_index] < 10:
            return None, None
    else:
        if cell_sum[target_index] < 10:
            return None, None

    fold_results = []
    for fold in range(n_kfold):
        true_data, predict_data = nihgcn_new(
            cell_exprs=exprs,
            drug_finger=drug_finger,
            res_mat=res,
            null_mask=null_mask,
            target_dim=dim,
            target_index=target_index,
            evaluate_fun=roc_auc,
            args=args,
            seed=seed,
        )
        fold_results.append((true_data, predict_data))

    return fold_results


# 並列処理の実行
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()

for dim in target_dim:
    # 全タスクを事前に生成
    tasks = [
        (dim, target_index, seed, args)
        for seed, target_index in enumerate(np.arange(res.shape[dim]))
    ]

    # 並列実行（プログレスバー付き）
    results = Parallel(n_jobs=n_jobs, verbose=0, prefer="threads")(
        delayed(process_iteration)(*task)
        for task in tqdm(tasks, desc=f"Processing dim {dim}")
    )

    # 結果の結合
    for fold_results in results:
        if fold_results is None:
            continue
        for true_data, predict_data in fold_results:
            true_data_s = pd.concat(
                [true_data_s, translate_result(true_data)],
                ignore_index=True,
                copy=False,  # メモリ節約のため
            )
            predict_data_s = pd.concat(
                [predict_data_s, translate_result(predict_data)],
                ignore_index=True,
                copy=False,
            )

  0%|          | 0/1005 [00:00<?, ?it/s]

epoch:   0 loss:0.700185 auc:0.5419
epoch:  20 loss:0.491298 auc:0.6830
epoch:  40 loss:0.401541 auc:0.7679
epoch:  60 loss:0.355135 auc:0.7488
epoch:  80 loss:0.330321 auc:0.7201
epoch: 100 loss:0.315482 auc:0.7129
epoch: 120 loss:0.307494 auc:0.6411
epoch: 140 loss:0.301804 auc:0.6435
epoch: 160 loss:0.292706 auc:0.6567
epoch: 180 loss:0.286559 auc:0.6663
epoch: 200 loss:0.286285 auc:0.6292
epoch: 220 loss:0.283011 auc:0.6471
epoch: 240 loss:0.277579 auc:0.6531
epoch: 260 loss:0.274671 auc:0.6627
epoch: 280 loss:0.272912 auc:0.6471
epoch: 300 loss:0.272129 auc:0.6400
epoch: 320 loss:0.269189 auc:0.6519
epoch: 340 loss:0.269001 auc:0.6411
epoch: 360 loss:0.267121 auc:0.6495
epoch: 380 loss:0.265749 auc:0.6220
epoch: 400 loss:0.263722 auc:0.6435
epoch: 420 loss:0.261562 auc:0.6352
epoch: 440 loss:0.262143 auc:0.6077
epoch: 460 loss:0.259877 auc:0.6531
epoch: 480 loss:0.257368 auc:0.6292
epoch: 500 loss:0.257185 auc:0.6220
epoch: 520 loss:0.255996 auc:0.6280
epoch: 540 loss:0.253974 auc

  0%|          | 1/1005 [00:11<3:15:31, 11.68s/it]

Fit finished.
epoch:   0 loss:0.702447 auc:0.6411
epoch:  20 loss:0.493299 auc:0.7474
epoch:  40 loss:0.403994 auc:0.7943
epoch:  60 loss:0.357958 auc:0.7886
epoch:  80 loss:0.332594 auc:0.8069
epoch: 100 loss:0.315118 auc:0.8194
epoch: 120 loss:0.306567 auc:0.7863
epoch: 140 loss:0.298456 auc:0.7806
epoch: 160 loss:0.292747 auc:0.7520
epoch: 180 loss:0.288588 auc:0.8023
epoch: 200 loss:0.284575 auc:0.6766
epoch: 220 loss:0.282052 auc:0.7360
epoch: 240 loss:0.279698 auc:0.5817
epoch: 260 loss:0.277327 auc:0.7051
epoch: 280 loss:0.274127 auc:0.7337
epoch: 300 loss:0.271959 auc:0.7954
epoch: 320 loss:0.271154 auc:0.6594
epoch: 340 loss:0.269301 auc:0.6674
epoch: 360 loss:0.267755 auc:0.5771
epoch: 380 loss:0.266264 auc:0.5794
epoch: 400 loss:0.264500 auc:0.7440
epoch: 420 loss:0.263889 auc:0.5851
epoch: 440 loss:0.262798 auc:0.7211
epoch: 460 loss:0.259457 auc:0.7326
epoch: 480 loss:0.258480 auc:0.6343
epoch: 500 loss:0.256787 auc:0.7109
epoch: 520 loss:0.256102 auc:0.7440
epoch: 540 los

  0%|          | 2/1005 [00:17<2:18:40,  8.30s/it]

...

Fit finished.
epoch:   0 loss:0.699378 auc:0.4062
epoch:  20 loss:0.488842 auc:0.8477
epoch:  40 loss:0.400909 auc:0.8164
epoch:  60 loss:0.355939 auc:0.7500
epoch:  80 loss:0.335401 auc:0.7109
epoch: 100 loss:0.315559 auc:0.7188
epoch: 120 loss:0.304036 auc:0.6875
epoch: 140 loss:0.296617 auc:0.6641
epoch: 160 loss:0.291909 auc:0.6133
epoch: 180 loss:0.289194 auc:0.6602
epoch: 200 loss:0.285284 auc:0.6328
epoch: 220 loss:0.286109 auc:0.6094
epoch: 240 loss:0.277137 auc:0.6133
epoch: 260 loss:0.274955 auc:0.6094
epoch: 280 loss:0.273865 auc:0.6016
epoch: 300 loss:0.274035 auc:0.5781
epoch: 320 loss:0.269156 auc:0.5781
epoch: 340 loss:0.267614 auc:0.5977
epoch: 360 loss:0.268114 auc:0.5781
epoch: 380 loss:0.266258 auc:0.5859
epoch: 400 loss:0.265499 auc:0.5586
epoch: 420 loss:0.263056 auc:0.5078
epoch: 440 loss:0.261354 auc:0.5391
epoch: 460 loss:0.260551 auc:0.5664
epoch: 480 loss:0.258745 auc:0.5508
epoch: 500 loss:0.259315 auc:0.5078
epoch: 520 loss:0.255999 auc:0.5195
epoch: 540 los

 11%|█         | 109/1005 [40:11<11:58:42, 48.13s/it]

Fit finished.
epoch:   0 loss:0.697218 auc:0.6374
epoch:  20 loss:0.490674 auc:0.8266
epoch:  40 loss:0.400985 auc:0.5861
epoch:  60 loss:0.355766 auc:0.4579
epoch:  80 loss:0.332811 auc:0.4274
epoch: 100 loss:0.317863 auc:0.4078
epoch: 120 loss:0.303864 auc:0.3785
epoch: 140 loss:0.296187 auc:0.4274
epoch: 160 loss:0.292647 auc:0.4420
epoch: 180 loss:0.289263 auc:0.4371
epoch: 200 loss:0.283576 auc:0.4078
epoch: 220 loss:0.286497 auc:0.3968
epoch: 240 loss:0.278253 auc:0.4066
epoch: 260 loss:0.277858 auc:0.3980
epoch: 280 loss:0.274310 auc:0.3846
epoch: 300 loss:0.274130 auc:0.4591
epoch: 320 loss:0.272455 auc:0.3858
epoch: 340 loss:0.268869 auc:0.3956
epoch: 360 loss:0.265603 auc:0.3663
epoch: 380 loss:0.266763 auc:0.4176
epoch: 400 loss:0.264660 auc:0.3529
epoch: 420 loss:0.264072 auc:0.4103
epoch: 440 loss:0.261873 auc:0.3907
epoch: 460 loss:0.261061 auc:0.3980
epoch: 480 loss:0.256500 auc:0.3773
epoch: 500 loss:0.256885 auc:0.4298
epoch: 520 loss:0.255506 auc:0.4115
epoch: 540 los

 11%|█         | 110/1005 [41:01<12:06:58, 48.74s/it]

Fit finished.
epoch:   0 loss:0.701434 auc:0.3900
epoch:  20 loss:0.489875 auc:0.6600
epoch:  40 loss:0.400542 auc:0.6400
epoch:  60 loss:0.357584 auc:0.6400
epoch:  80 loss:0.332851 auc:0.6300
epoch: 100 loss:0.315878 auc:0.6200
epoch: 120 loss:0.304897 auc:0.6300
epoch: 140 loss:0.298907 auc:0.6600
epoch: 160 loss:0.293923 auc:0.7000
epoch: 180 loss:0.287913 auc:0.7300
epoch: 200 loss:0.285511 auc:0.7200
epoch: 220 loss:0.282194 auc:0.7100
epoch: 240 loss:0.278629 auc:0.6900
epoch: 260 loss:0.276482 auc:0.7100
epoch: 280 loss:0.273528 auc:0.6500


In [ ]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")